In [ ]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (78.0 kB/s)
Reading package li

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-13 02:00:26--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.51MB/s    in 0.2s    

2022-11-13 02:00:26 (5.51 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in amazon reviews from S3 into a DataFrame
#url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [ ]:
# Get all columns to adjust dataframe
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Create a unique product dataframe
#unique_products = df['product_id'].nunique


In [ ]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

960872
960679
960679


In [ ]:
# Create review table dataframe
from pyspark.sql.functions import *

review_df = df.select(["review_id", "customer_id", "product_id", "product_parent", date_format(df['review_date'],"yyyy-MM-dd").alias("yyyy-MM-dd")])
review_df.show(5)



+--------------+-----------+----------+--------------+----------+
|     review_id|customer_id|product_id|product_parent|yyyy-MM-dd|
+--------------+-----------+----------+--------------+----------+
|R1006OPQYUMAHS|   43765005|B0000TIIZ0|     529990321|2008-03-05|
|R1010XUX2AIF10|   44007561|B004KTK2M8|      38406392|2013-04-11|
|R102DONPK6YEGN|   52805367|B00NHEU9U2|      33550162|2015-06-23|
|R102N48HXFM9WY|   18449245|B000PSQXWG|     517553139|2013-02-04|
|R1060WX8UX26JV|   18106128|B0045H3K0G|     989815375|2013-04-15|
+--------------+-----------+----------+--------------+----------+
only showing top 5 rows



In [ ]:
#from pyspark.sql.functions import *

#df=spark.createDataFrame([["1"]],["id"])
#df.select(date_format(df['review_date'],"yyyy-MM-dd").alias("yyyy-MM-dd")).show(5)

In [ ]:
review_df_new = review_df.withColumnRenamed("review_id", "review_id") \
        .withColumnRenamed("customer_id", "customer_id") \
        .withColumnRenamed("product_id", "product_id") \
        .withColumnRenamed("product_parent", "product_parent") \
        .withColumnRenamed("yyyy-MM-dd", "review_date")
review_df_new.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1006OPQYUMAHS|   43765005|B0000TIIZ0|     529990321| 2008-03-05|
|R1010XUX2AIF10|   44007561|B004KTK2M8|      38406392| 2013-04-11|
|R102DONPK6YEGN|   52805367|B00NHEU9U2|      33550162| 2015-06-23|
|R102N48HXFM9WY|   18449245|B000PSQXWG|     517553139| 2013-02-04|
|R1060WX8UX26JV|   18106128|B0045H3K0G|     989815375| 2013-04-15|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- yyyy-MM-dd: string (nullable = true)



In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://bootcampdemo.cjhuht28cggy.us-east-1.rds.amazonaws.com:5432/postgres"

config = {"user":"postgres",
          "password": "password",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review table DataFrame to table

review_df_new.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)